Imports

In [1]:
import numpy as np
import pandas as pd
print('Imports are ready!')

Imports are ready!


 # **Loading Datasets**

 As you can see in both sets we contain 20k examples, each containing it's information of its "ID_code" and 200 numerical features. It is important to note that the training set contains an aditional column containing the binary target for each example

In [2]:
print("Let us load the training and the test set, please wait.")
train=pd.read_csv('/Users/juansmacbook/PycharmProjects/Santander_Transaction/santander_data/train.csv')
test=pd.read_csv('/Users/juansmacbook/PycharmProjects/Santander_Transaction/santander_data/test.csv')
print("Shape of training set: "+str(train.shape))
print("Shape of testing set: "+str(test.shape))

Let us load the training and the test set, please wait.
Shape of training set: (200000, 202)
Shape of testing set: (200000, 201)


# **"Magic" and synthetic examples in the test dataset**
This competition is interesting because people in the forums discovered that the 50% of the test set is synthetic! The key to distinguish the real testing examples from the fake ones is checking whether the value of a certain feature isn't repeated in another training example. Therefore, only the examples that have *at least one unique value in any of their features* are considered to be real!

This of course wasn't discovered by me, this is what the community calls "Magic" when it comes to kaggle competitions. I will elaborate further on why this is really important for the model.

In this order of ideas, my next task will be to filter out the fake testing examples and prove to you that this is actually the case. We should be able to notice that 10.000  training examples are real and 10.000 are fake!

In [3]:
"""Here we define a function that inputs a column from a data frame and returns a list of the values that only appear one time in that data set"""

def unique_values(column):
    count=column.value_counts()
    return count.index[count==1]

"""We are going to check what examples in test set are real or not by doing value counts, examples whose variables arent repeated in the whole test set are considered to be real!"""

def feature_uniqueness(data_frame=test): # This function inputs the TEST DATASET and returns its "extended" version that says which feature is unique

    for column in data_frame.columns[1:]: #Notice that we are dropping the first column since we don't care bout ID
        new_column=pd.Series(
            data=data_frame[column].isin(unique_values(data_frame[column])).to_numpy(),
            name=column+'_unique?'
        )
        data_frame=pd.concat([data_frame,new_column],axis=1)
    return data_frame

In [5]:
"""This is our new data frame containing 200 extra features that tell us which feature is unique in each column for every training example"""
temporal_df=feature_uniqueness()
temporal_df.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190_unique?,var_191_unique?,var_192_unique?,var_193_unique?,var_194_unique?,var_195_unique?,var_196_unique?,var_197_unique?,var_198_unique?,var_199_unique?
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,False,False,False,False,False,False,False,False,False,False
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,False,False,False,False,False,False,False,False,False,False
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,False,False,False,False,False,False,False,False,False,False
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,False,False,False,False,False,False,True,True,False,False
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,False,False,False,False,False,False,False,False,False,False


,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190_unique?,var_191_unique?,var_192_unique?,var_193_unique?,var_194_unique?,var_195_unique?,var_196_unique?,var_197_unique?,var_198_unique?,var_199_unique?
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,False,False,False,False,False,False,False,False,False,False
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,False,False,False,False,False,False,False,False,False,False
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,False,False,False,False,False,False,False,False,False,False
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,False,False,False,False,False,False,True,True,False,False
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,False,False,False,False,False,False,False,False,False,False


# **Proof that 50% of data set is fake!**
Just as previously mentioned, the only examples that are we going to consider to be non-synthetic will be those have at least one unique value on their features, the ones that don't have any will be considered as fake. We can easily filter out the real and the fake ones via a simple condition.
Using this condition we are going to split the training set into real and fake examples

In [13]:
#In each example we are checking whether if any of their lats 200 features has a true statement
condition=temporal_df[temporal_df.columns[201:]].any(axis=1)
condition

0         False
1         False
2         False
3          True
4         False
          ...  
199995     True
199996     True
199997    False
199998    False
199999     True
Length: 200000, dtype: bool

In [9]:

print('_.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._')
print('Just as mentioned in the forums, 50% of test data is fake!')
print(
    temporal_df[temporal_df.columns[201:]].any(axis=1).value_counts()
)
print('_.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._')


#Now we can split the dataset in the real part and the fake part

real_test_examples==temporal_df[condition==True]
fake_test_examples=temporal_df[condition==False]

#let us delete x_test to save some memory since we already have it splitted in two parts
del temporal_df


_.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._
Just as mentioned in the forums, 50% of test data is fake!
False    100000
True     100000
dtype: int64
_.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._


In [8]:
#We create a new variable that says if an example is real or not

temporal_df.drop(x_test.columns[201:401], axis=1,
                 inplace=True#Cambiar por TRUE DESPUESSSS
                 )
temporal_df[7:8]

NameError: name 'x_test' is not defined

In [9]:




# We are going to concatenate the train set and the real training examples into x_train to do a value count later
# We check what values are uniques in both training set and real training set
x_train=pd.concat( [train.drop(['target'],axis=1), real_test_examples.drop(['real'], axis=1)] , axis=0)
x_train.reset_index(inplace=True) #lets reset index to avoid problems later
x_train.drop(['index'], axis=1, inplace=True)
print('Shape after concatenating: '+str(x_train.shape))
# We append to the dataset 200 new features containing what values are unique
for column in x_train.columns[1:]:
    uniques_count=x_train[column].value_counts().to_dict()
    x_train[column+'_unique']=x_train[column].apply(lambda x: 1 if uniques_count[x]==1 else 0).values
    fake_test_examples[column+'_unique']= 0
print('Finished!')
#We create the final test set containing all unique value counts!!!!!!
fake_test_examples.drop(['real'], axis=1, inplace=True)
x_test=pd.concat([x_train[:][200000:],fake_test_examples])

#We create the final training set containing value counts!
x_train=x_train[:][:200000]


print('Ready!')

/var/folders/13/z6c3b3d50nnbcf08brkd87dw0000gn/T/ipykernel_20316/2412576041.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  x_test[col+'_unique']= x_test[col].isin( unique_values(x_test[col]) )
/var/folders/13/z6c3b3d50nnbcf08brkd87dw0000gn/T/ipykernel_20316/2412576041.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  x_test[col+'_unique']= x_test[col].isin( unique_values(x_test[col]) )
/var/folders/13/z6c3b3d50nnbcf08brkd87dw0000gn/T/ipykernel_20316/2412576041.py:9: PerformanceWarning: DataFrame is highly fragmented.  Thi

/////////////////////////////////////////////////////////
Just as mentioned in the forums, 50% of test data is fake!
False    100000
True     100000
Name: real, dtype: int64
/////////////////////////////////////////////////////////
/////////////////////////////////////////////////////////
/////////////////////////////////////////////////////////
Shape after concatenating: (300000, 201)


/var/folders/13/z6c3b3d50nnbcf08brkd87dw0000gn/T/ipykernel_20316/2412576041.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_test_examples[column+'_unique']= 0
/var/folders/13/z6c3b3d50nnbcf08brkd87dw0000gn/T/ipykernel_20316/2412576041.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_test_examples[column+'_unique']= 0
/var/folders/13/z6c3b3d50nnbcf08brkd87dw0000gn/T/ipykernel_20316/2412576041.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

KeyboardInterrupt: 